# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [189]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.utils import normalize
from sklearn.model_selection import train_test_split

In [190]:
# your code here
df = pd.read_csv('tic-tac-toe.csv')

In [191]:
df = pd.DataFrame(df.replace('x',1).replace('o',0).replace('b',0.5))

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [192]:
df

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,True
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,True
2,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,True
3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.5,0.5,True
4,1.0,1.0,1.0,1.0,0.0,0.0,0.5,0.0,0.5,True
...,...,...,...,...,...,...,...,...,...,...
953,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,False
954,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,False
955,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,False
956,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,False


In [193]:
# your code here
X = np.array(df.drop(columns=['class']))
y = np.array(df['class'])

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 29)

In [195]:
X_train.shape

(766, 9)

In [196]:
X_train = normalize(X_train,axis=1)
X_test = normalize(X_test,axis=1)

In [209]:

model = Sequential()
model.add(Dense(82, activation='relu'))
model.add(Dense(82, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=10)

Epoch 1/10
766/766 [==============================] - 0s 216us/step - loss: 0.6242 - accuracy: 0.6580
Epoch 2/10
766/766 [==============================] - 0s 76us/step - loss: 0.5825 - accuracy: 0.6997
Epoch 3/10
766/766 [==============================] - 0s 64us/step - loss: 0.5482 - accuracy: 0.7441
Epoch 4/10
766/766 [==============================] - 0s 64us/step - loss: 0.5293 - accuracy: 0.7389
Epoch 5/10
766/766 [==============================] - 0s 76us/step - loss: 0.5089 - accuracy: 0.7611
Epoch 6/10
766/766 [==============================] - 0s 65us/step - loss: 0.4919 - accuracy: 0.7546
Epoch 7/10
766/766 [==============================] - 0s 64us/step - loss: 0.4745 - accuracy: 0.7781
Epoch 8/10
766/766 [==============================] - 0s 63us/step - loss: 0.4550 - accuracy: 0.7807
Epoch 9/10
766/766 [==============================] - 0s 64us/step - loss: 0.4294 - accuracy: 0.8120
Epoch 10/10
766/766 [==============================] - 0s 62us/step - loss: 0.4042 - accur

In [210]:
model.summary()

Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_139 (Dense)            (None, 82)                820       
_________________________________________________________________
dense_140 (Dense)            (None, 82)                6806      
_________________________________________________________________
dense_141 (Dense)            (None, 2)                 166       
Total params: 7,792
Trainable params: 7,792
Non-trainable params: 0
_________________________________________________________________


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [211]:
# your code here
predictions = model.predict(X_test[:5])
print(np.argmax(predictions, axis=1))

# Check our predictions against the ground truths.
print(y_test[:5]) 

[0 1 0 1 0]
[False  True False  True False]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [224]:
# your code here
model = Sequential()
model.add(Dense(82, activation='relu'))
model.add(Dense(82, activation='relu'))
model.add(Dense(82, activation='relu'))
model.add(Dense(82, activation='relu'))
model.add(Dense(82, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=7)

Epoch 1/7
766/766 [==============================] - 1s 666us/step - loss: 0.6338 - accuracy: 0.6580
Epoch 2/7
766/766 [==============================] - 0s 223us/step - loss: 0.5618 - accuracy: 0.7102
Epoch 3/7
766/766 [==============================] - 0s 125us/step - loss: 0.4958 - accuracy: 0.7546
Epoch 4/7
766/766 [==============================] - 0s 104us/step - loss: 0.4088 - accuracy: 0.8068
Epoch 5/7
766/766 [==============================] - 0s 121us/step - loss: 0.2741 - accuracy: 0.8916
Epoch 6/7
766/766 [==============================] - 0s 97us/step - loss: 0.1326 - accuracy: 0.9726
Epoch 7/7
766/766 [==============================] - 0s 120us/step - loss: 0.0602 - accuracy: 0.9883


**Which approach(es) did you find helpful to improve your model performance?**

In [225]:
# your answer here
predictions = model.predict(X_test[:10])
print(np.argmax(predictions, axis=1))
print(y_test[:10])
#predictions

[0 1 0 1 0 0 0 1 0 0]
[False  True False  True False False False  True False False]
